In [72]:
# librerías de manejo de datos y algebra lineal
import pandas as pd
import numpy as np

# librerías dedicadas a machine learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler

# quitamos las warnings para mejorar la estética del código
import warnings
warnings.filterwarnings('ignore')

In [73]:
# importamos los datos
data = pd.read_csv('Cleaned_data_for_model.csv', sep=',')
data = data.drop(columns='Unnamed: 0')

In [74]:
# mostramos la información general del dataframe
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99499 entries, 0 to 99498
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   property_type  99499 non-null  object 
 1   price          99499 non-null  int64  
 2   location       99499 non-null  object 
 3   city           99499 non-null  object 
 4   baths          99499 non-null  int64  
 5   purpose        99499 non-null  object 
 6   bedrooms       99499 non-null  int64  
 7   Area_in_Marla  99499 non-null  float64
dtypes: float64(1), int64(3), object(4)
memory usage: 6.1+ MB


In [92]:
# mostramos la información más específica de los datos
data.describe()

,price,baths,bedrooms,area,density
count,9.919500e+04,99195.000000,99195.000000,99195.000000,99195.000000
mean,1.038573e+07,3.532083,3.360653,8.764871,1.040235
std,1.066463e+07,1.473221,1.244423,7.468682,0.579598
min,1.550000e+04,1.000000,1.000000,0.200000,0.011111
25%,1.500000e+05,2.000000,2.000000,4.800000,0.650000
50%,7.500000e+06,3.000000,3.000000,6.800000,0.952381
75%,1.560000e+07,5.000000,4.000000,10.000000,1.250000
max,4.490000e+07,7.000000,6.000000,194.000000,20.000000


In [75]:
# limpiamos columnas irrelevantes y cambiamos nombres por facilidad
data = data.drop(columns=['purpose', 'location']).rename(columns={'Area_in_Marla' : 'area'})
data

,property_type,price,city,baths,bedrooms,area
0,Flat,10000000,Islamabad,2,2,4.0
1,Flat,6900000,Islamabad,3,3,5.6
2,House,16500000,Islamabad,6,5,8.0
3,House,43500000,Islamabad,4,4,40.0
4,House,7000000,Islamabad,3,3,8.0
...,...,...,...,...,...,...
99494,Flat,7500000,Karachi,3,3,8.0
99495,House,8800000,Karachi,4,3,8.0
99496,House,14000000,Karachi,3,3,8.0
99497,House,14000000,Karachi,4,4,14.0


In [77]:
# combinamos variables y las limpiamos
data = data[(data.baths != 0) & (data.bedrooms != 0) & (data.area != 0)]
data['density'] = (data['baths'] + data['bedrooms'])/data['area']

In [79]:
# eliminamos outliers para hacer más preciso el modelo 
q1 = data['price'].quantile(0.25)
q3 = data['price'].quantile(0.75)
iqr = q3 - q1
outliers = data[(data['price'] < q1 - 1.5*iqr) | (data['price'] > q3 + 1.5*iqr)]
data_clean = data[(data['price'] >= q1 - 1.5*iqr) | (data['price'] <= q3 + 1.5*iqr)]

In [80]:
# dividimos en datos a tomar en cuenta y los targets
X = data_clean.drop(columns='price')
y = data_clean['price']

In [81]:
X['property_type_coded'] = X['property_type'].astype('category').cat.codes
X['city_coded'] = X['city'].astype('category').cat.codes

In [82]:
# escalamos los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X[['density',
       'property_type_coded', 'city_coded']])

In [84]:
# creamos el modelo
rfreg = RandomForestRegressor(
    n_estimators=500,
    max_depth=5,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42
)

In [85]:
# ajustamos el modelo mediante cross_val_score
scores = cross_val_score(rfreg, X_scaled, y, cv=5, scoring='neg_mean_squared_error')
rmse_scores = np.sqrt(-scores)

In [90]:
# encontramos la media del neg_mean_squared_error 
rmse_scores.mean()

8325012.990524577

In [91]:
# vemos el rango de precios limpios
print(data_clean['price'].min(), data_clean['price'].max())

15500 44900000


### Conclusión

El error mostrado anteriormente es menor al valor máximo dentro del rango de los precios, lo que lo convierte en un error relativamente aceptable. Este desarrollo es bastante simple, pero podría enriquecerse con técnicas adicionales, como **feature engineering**, **selección de características** (*feature selection*) y una mejora en la detección de valores atípicos (*outliers*) mediante métodos alternativos.

A pesar de su simplicidad, los resultados obtenidos son bastante buenos y permiten predecir, con una precisión considerable, los precios de los inmuebles basándose en variables como el tipo de propiedad, la densidad ($\text{baths} + \text{bedrooms} / \text{area}$) y la ciudad.

El conjunto de datos utilizado fue extraído de **Kaggle**, específicamente del [House Prices 2023 Dataset](https://www.kaggle.com/datasets/howisusmanali/house-prices-2023-dataset). Este dataset fue elegido porque, al trabajar en un banco con estrictas políticas de confidencialidad, no es posible utilizar datos reales.
